In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt


data_dir = r'C:\Users\advai\OneDrive\Desktop\python_ws\model flipkart\newapple'

# Check if the directory is correct
if os.path.exists(data_dir):
    print("Data directory exists. Subdirectories are:")
    print(os.listdir(data_dir))  # List the subfolders (shelf life categories)
else:
    print("Directory not found!")


Data directory exists. Subdirectories are:
['Apple(1-5)', 'Apple(10-14)', 'Apple(5-10)', 'banana.jpg', 'Expired', 'frame21.jpg', 'frame484.jpg', 'istockphoto-1198706343-612x612.jpg', 'nuripp - Copy.jpg', 'nuripp.jpg', 'rippapple.jpg', 'Screenshot 2024-09-28 160615.png', 'yellow-mango.jpeg']


In [3]:
# Define data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% training, 20% validation
)

# Training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Display the class mapping
class_labels = train_generator.class_indices
print("Class Labels:", class_labels)


Found 845 images belonging to 4 classes.
Found 209 images belonging to 4 classes.
Class Labels: {'Apple(1-5)': 0, 'Apple(10-14)': 1, 'Apple(5-10)': 2, 'Expired': 3}


In [4]:
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential()

# First Conv block
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second Conv block
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third Conv block
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Fourth Conv block
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten and Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(len(class_labels), activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


c:\Users\advai\anaconda3\envs\TradeTravern\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,454,660 (13.18 MB)

 Trainable params: 3,454,660 (13.18 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Set the number of epochs
epochs = 30

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Display accuracy after training
training_acc = history.history['accuracy'][-1]
validation_acc = history.history['val_accuracy'][-1]

print(f'Training Accuracy: {training_acc * 100:.2f}%')
print(f'Validation Accuracy: {validation_acc * 100:.2f}%')


Epoch 1/30


c:\Users\advai\anaconda3\envs\TradeTravern\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 668ms/step - accuracy: 0.4076 - loss: 1.2717 - val_accuracy: 0.6562 - val_loss: 0.7259
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4375 - loss: 1.1266 - val_accuracy: 0.8235 - val_loss: 0.5646
Epoch 3/30


c:\Users\advai\anaconda3\envs\TradeTravern\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 387ms/step - accuracy: 0.6898 - loss: 0.6780 - val_accuracy: 0.5833 - val_loss: 0.8210
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6875 - loss: 0.5753 - val_accuracy: 0.8824 - val_loss: 0.2863
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 374ms/step - accuracy: 0.7293 - loss: 0.5772 - val_accuracy: 0.7969 - val_loss: 0.6117
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6562 - loss: 0.7013 - val_accuracy: 0.7059 - val_loss: 0.5137
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 10s 377ms/step - accuracy: 0.8026 - loss: 0.4395 - val_accuracy: 0.8958 - val_loss: 0.3368
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7812 - loss: 0.3880 - val_accuracy: 0.8824 - val_loss: 0.5103
Epoch 9/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 379ms/step - accuracy: 0.8326 - loss: 0.3692 - val_accuracy: 0.7500 - val_loss: 0.4757
Epoch 10/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8125 - loss: 0.3491 - val_accuracy: 0.8824 - val_los

In [1]:
### or 
def predict_and_display_multiple(img_paths, model, class_indices):
    """
    Predicts the shelf life of multiple apple images and displays each image with prediction.
    
    Parameters:
    - img_paths (list): List of image file paths.
    - model: Trained Keras model.
    - class_indices (dict): Mapping of class indices to class names and shelf life.
    
    Returns:
    - None: Prints prediction details and displays each image.
    """
    for img_path in img_paths:
        print(f'Processing image: {img_path}')
        predict_and_display(img_path, model, class_indices)
        print('-' * 50)  # Separator between images

# Example: Predict on multiple images
image_list = [
    r'C:\Users\advai\OneDrive\Desktop\python_ws\model flipkart\newapple\frame484.jpg',
    r'C:\Users\advai\OneDrive\Desktop\python_ws\model flipkart\newapple\frame21',
    r'C:\Users\advai\OneDrive\Desktop\python_ws\model flipkart\newapple\Screenshot 2024-09-28 160615.jpg'
]

# Call the multiple prediction and display function
predict_and_display_multiple(image_list, model, class_indices)


NameError: name 'model' is not defined